In [1]:

import sys
import os

repo_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd())))
sys.path.append(repo_root)

print(repo_root)

/home/jackyeung99/classes/proj_energy_volatility


In [2]:
import matplotlib.pyplot as plt
import math 
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
from statsmodels.tsa.stattools import adfuller, coint
from arch import arch_model

from src.trading.IBKR_trading import TradingApp

In [3]:
def realized_vol(time_series, window: int = 21, annualize: bool = True) -> pd.DataFrame:
    """Compute rolling standard deviation of daily returns (realized volatility)."""
    
    volatility = time_series.rolling(window).std()
    
    if annualize:
        volatility *= np.sqrt(256)
    
    return volatility

def long_to_wide(df, pivot_by='close'):

    wide = df.pivot(index='date', columns='Symbol', values=pivot_by)
    wide = wide.reset_index()  
    wide.columns.name = None

    return wide

In [4]:
data = ['CPIAUCSL', 'GDPC1', 'CNP16OV', 'FEDFUNDS']

In [5]:
HOST = os.getenv("IB_HOST")
PORT = int(os.getenv("IB_PORT"))
CLIENT_ID = int(os.getenv("IB_CLIENT_ID"))

app = TradingApp()
await app.start(HOST, PORT, clientid=CLIENT_ID)

Connected to Interactive Brokers successfully.


ERROR:ib_insync.wrapper:Error 321, reqId 9: Error validating request.-'bL' : cause - When specifying a unit, historical data request duration format is integer{SPACE}unit (S|D|W|M|Y)., contract: Stock(conId=4215217, symbol='XLE', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='XLE', tradingClass='XLE')
ERROR:ib_insync.wrapper:Error 321, reqId 10: Error validating request.-'bL' : cause - When specifying a unit, historical data request duration format is integer{SPACE}unit (S|D|W|M|Y)., contract: Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')


In [6]:

df = pdr.DataReader(
    data,
    "fred",
    start="1980-01-01",
    end="2025-07-01"
).astype(float).reset_index()

In [7]:
df['INF'] = 400 * np.log(df['CPIAUCSL']/ df['CPIAUCSL'].shift(-1))
df['INF_demeaned'] = df['INF'] - df['INF'].mean()
df['INF_r'] = df['INF'].diff()

In [23]:
index = ['XLE', 'SPY']
index_stock = await app.fetch_many_symbols(index, '10 Y', '1 M', 'TRADES', rth=True)
wide = long_to_wide(index_stock)

In [24]:
wide

,date,SPY,XLE
0,2015-11-30,208.69,68.02
1,2015-12-31,203.87,60.32
2,2016-01-29,193.72,58.21
3,2016-02-29,193.56,56.58
4,2016-03-31,205.52,61.89
...,...,...,...
116,2025-07-31,632.08,87.21
117,2025-08-29,645.05,90.39
118,2025-09-30,666.18,89.34
119,2025-10-31,682.06,88.13


In [27]:
df

,DATE,CPIAUCSL,GDPC1,CNP16OV,FEDFUNDS,INF,INF_demeaned,INF_r,vol
0,1980-01-01,78.000,7341.557,166544.0,13.82,-5.095610,-4.056597,NaN,NaN
1,1980-02-01,79.000,NaN,166759.0,14.13,-5.531201,-4.492188,-0.435590,NaN
2,1980-03-01,80.100,NaN,166984.0,17.19,-3.975188,-2.936175,1.556013,NaN
3,1980-04-01,80.900,7190.289,167197.0,17.61,-3.936071,-2.897058,0.039117,NaN
4,1980-05-01,81.700,NaN,167407.0,10.98,-3.897717,-2.858704,0.038355,NaN
...,...,...,...,...,...,...,...,...,...
542,2025-03-01,319.615,NaN,273023.0,4.33,-0.882589,0.156424,-1.082779,10.746972
543,2025-04-01,320.321,23770.976,273197.0,4.33,-0.323295,0.715718,0.559294,9.779060
544,2025-05-01,320.580,NaN,273385.0,4.33,-1.146275,-0.107263,-0.822981,10.197592
545,2025-06-01,321.500,NaN,273585.0,4.33,-0.785542,0.253471,0.360733,9.258413
